<a href="https://colab.research.google.com/github/kylebrechin/DS-Unit-1-Sprint-2-Statistics/blob/master/module2/LS_DS_122_Chi2_Tests_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment - Practice Chi^2 Tests

Use the following dataset relating to math scores of students in two different Portugese schools:

<https://archive.ics.uci.edu/ml/datasets/Student+Performance>

### 1) Load the dataset specifically relating to math scores as a new dataframe.

There are two datasets in the `student.zip` file, make sure you use `student-mat.csv`.


In [1]:
# YOUR WORK HERE
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip

--2020-07-27 06:39:01--  https://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20478 (20K) [application/x-httpd-php]
Saving to: ‘student.zip’

student.zip         100%[===================>]  20.00K  --.-KB/s    in 0.1s    

2020-07-27 06:39:02 (147 KB/s) - ‘student.zip’ saved [20478/20478]



In [2]:
!unzip student.zip

Archive:  student.zip
  inflating: student-mat.csv         
  inflating: student-por.csv         
  inflating: student-merge.R         
  inflating: student.txt             


In [16]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

students = pd.read_csv('student-mat.csv', sep=';')
students.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,3,yes,no,yes,no,yes,yes,yes,no,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,yes,yes,yes,yes,yes,yes,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,yes,no,yes,yes,no,no,4,3,2,1,2,5,4,6,10,10


In [25]:
students.describe(exclude='number')

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
count,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395,395
unique,2,2,2,2,2,5,5,4,3,2,2,2,2,2,2,2,2
top,GP,F,U,GT3,T,other,other,course,mother,no,yes,no,yes,yes,yes,yes,no
freq,349,208,307,281,354,141,217,145,273,344,242,214,201,314,375,329,263


### 2) Use Chi^2 tests and `stats.chi2_contingency()` to identify:
 - Two pairs of variables that are dependent (are associated with one another).
 - Two pairs of variables that are independent (have no significant relationship).

Does it make intuitive sense why the variables in these pairs might or might not show a relationship to one another? 


#### Independent
Age and study time
Travel time and study time

In [18]:
# compare age and study time
age_study = pd.crosstab(students['age'], students['studytime'])
chi2_contingency(age_study)[1]

0.45863728792691505

In [19]:
# compare study and travel time
travel_study = pd.crosstab(students['traveltime'], students['studytime'])
chi2_contingency(travel_study)[1]

0.26223746329595793

In [21]:
# compare failures and study time
study_failure = pd.crosstab(students['studytime'], students['failures'])
chi2_contingency(study_failure)[1]

0.06258448399973988

In [35]:
# absences range from 0 - 75
# put them in groups of 0, 1, 2-6, 7+
students['abs_group']= 0
students.loc[students['absences'] > 1, 'abs_group'] = 1
students.loc[students['absences'] > 2, 'abs_group'] = 2
students.loc[students['absences'] > 7, 'abs_group'] = 3

students['abs_group'].value_counts().sort_index()

0    118
1     65
2    104
3    108
Name: abs_group, dtype: int64

In [37]:
# compare failures and absences
absent_failures = pd.crosstab(students['abs_group'], students['failures'])
chi2_contingency(absent_failures)[1]

0.05412494596626286

In [23]:
# compare sex and failures
sex_failures = pd.crosstab(students['sex'], students['failures'])
chi2_contingency(sex_failures)[1]

0.31685806856495097

In [38]:
# compare sex and absences
sex_absent = pd.crosstab(students['sex'], students['abs_group'])
chi2_contingency(sex_absent)[1]

0.8929189470457711

In [26]:
# compare sex and activities
sex_activities = pd.crosstab(students['sex'], students['activities'])
chi2_contingency(sex_activities)[1]

0.05965215481197171

In [27]:
# compare freetime and activities
free_activities = pd.crosstab(students['freetime'], students['activities'])
chi2_contingency(free_activities)[1]

0.26864678298146993

In [28]:
# compare parent status and failures
pareents_failures = pd.crosstab(students['Pstatus'], students['failures'])
chi2_contingency(pareents_failures)[1]

0.9340177102114694

### 3) Use NumPy to perform your own chi^2 test "from scratch" 

Pick any of the chi^2 tests that you ran in part 2 and try them on your own. You should get the same results that Scipy got for all four values returned from `chi2_contingency()`

In [ ]:
# YOUR WORK HERE

## Stretch goals:

### 1. Refactor your code so it is elegant, readable, and holds reusable code in functions.

In [ ]:
# YOUR WORK HERE



### 2. Check For Understanding - Study and write your own explanations/definitions for these topics:

- What is a sample "estimate" in statistics?

- What are hypothesis test? How are they useful?

- What is a "null hypothesis?"

- What is a p-value and what does it represent?

- What does it mean for something to be "statistically significant?"

- What is a test statistic and how does it relate to a p-value?

- What are "degrees of freedom" and how are they calculated in a 1-sample, 2-sample, and chi^2 test for independence?

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)